In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import eigh


In [2]:
def pauli_x():
    """Pauli X matrix."""
    return np.array([[0, 1], [1, 0]])

def free_paramagnetic_hamiltonian(N, h=1.0):
    """
    Constructs the Hamiltonian for the free paramagnetic model.
    
    Parameters:
        N (int): Number of spins.
        h (float): Strength of the transverse field.
    
    Returns:
        H (numpy.ndarray): The Hamiltonian matrix.
    """
    I = np.eye(2)  # Identity matrix
    X = pauli_x()  # Pauli X matrix

    # Initialize Hamiltonian
    H = np.zeros((2**N, 2**N), dtype=np.float64)

    # Construct the sum of Pauli-X matrices
    for i in range(N):
        term = 1
        for j in range(N):
            if j == i:
                term = np.kron(term, X)  # Apply X at site i
            else:
                term = np.kron(term, I)  # Identity elsewhere
        H -= h * term  # Multiply by field strength and subtract

    return H

def entanglement_entropy(psi, subsystem_size, total_size):
    """
    Computes the bipartite entanglement entropy of a pure state.
    
    Parameters:
    psi : np.array
        The wavefunction (state vector) of the system.
    subsystem_size : int
        The number of qubits in subsystem A.
    total_size : int
        The total number of qubits in the system.
    
    Returns:
    float
        The von Neumann entanglement entropy S_A.
    """
    # Calculate the size of the environment
    environment_size = total_size // subsystem_size
    
    # Reshape psi into a 2^subsystem_size x environment_size matrix
    psi_matrix = psi.reshape((subsystem_size, environment_size))
    
    # Compute the reduced density matrix rho_A = Tr_B(|psi><psi|)
    rho_A = np.dot(psi_matrix, psi_matrix.conj().T)  # Partial trace over B
    
    # Compute eigenvalues of rho_A
    eigenvalues = np.linalg.eigvalsh(rho_A)
    
    # Filter out zero eigenvalues to avoid numerical issues in log calculation
    eigenvalues = eigenvalues[eigenvalues > 0]
    
    # Compute von Neumann entropy S_A = -Tr(rho_A log rho_A)
    entropy = -np.sum(eigenvalues * np.log2(eigenvalues))
    
    return entropy

In [3]:
N_spins = 12  # Number of spins
H_free = free_paramagnetic_hamiltonian(N_spins)

# Compute eigenvalues and eigenvectors for the free paramagnetic Hamiltonian
eigenvalues_free, eigenvectors_free = eigh(H_free)


# Compute entanglement entropy for all eigenstates in the free paramagnetic Hamiltonian
subsystem_size = len(eigenvectors_free[:, 0]) // 2
entropies_free = [entanglement_entropy(eigenvectors_free[:, i], subsystem_size, len(eigenvectors_free[:, i]))
                  for i in range(eigenvectors_free.shape[1])]

# Plot energy spectrum
plt.figure(figsize=(8, 6))
plt.plot(range(len(eigenvalues_free)), eigenvalues_free, 'o', markersize=3)
plt.xlabel('Eigenstate index')
plt.ylabel('Energy')
plt.title('Energy Spectrum of Free Paramagnetic Model')
plt.grid(True)
plt.show()

# Plot entanglement entropy
plt.figure(figsize=(8, 6))
plt.plot(eigenvalues_free, entropies_free, 'o', markersize=3)
plt.xlabel('Energy')
plt.ylabel('Entanglement Entropy')
plt.title('Entanglement Entropy vs Energy for Free Paramagnetic Model')
plt.grid(True)
plt.show()

# Construct Neel state |1010...⟩ for length N_spins
neel_state_pattern = tuple((i % 2 for i in range(N_spins)))
neel_state_vector = np.array([1 if format(i, f'0{N_spins}b') == ''.join(map(str, neel_state_pattern)) else 0 
                               for i in range(2**N_spins)], dtype=float)

# Compute overlaps with all eigenstates
overlaps_free = np.abs(np.dot(eigenvectors_free.T, neel_state_vector))**2

# Identify the eigenstate with maximum overlap
max_overlap_index = np.argmax(overlaps_free)
max_overlap_energy = eigenvalues_free[max_overlap_index]

# Plot Neel state overlap
plt.figure(figsize=(8, 6))
plt.plot(eigenvalues_free, overlaps_free, 'o', markersize=3)
plt.xlabel('Energy')
plt.ylabel('Neel State Overlap')
plt.title('Neel State Overlap vs Energy for Free Paramagnetic Model')
plt.grid(True)
plt.show()

# Display maximum overlap
max_overlap_free = overlaps_free[max_overlap_index]
max_overlap_free, max_overlap_energy

KeyboardInterrupt: 